In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False


### Make scene

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 3, -2, 2, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Planning scene

In [4]:
from pkg.geometry.geometry import *
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [5]:
pscene = PlanningScene(gscene, combined_robot=crob)
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE
BRUSH_NAME = "brush_face"
brush_face = add_indy_sweep_tool(gscene, ROBOT_NAME, face_name=BRUSH_NAME, tool_offset=0.12, color_col=(0.8,0.8,0.8,0.2))
brush_b = pscene.create_binder(BRUSH_NAME, BRUSH_NAME, WayFramer, point=(0,0,brush_face.dims[2]/2), rpy=(0,np.pi,0))
# from demo_utils.environment import add_kiro_indytool_down
# add_kiro_indytool_down(gscene, zoff=0, tool_link="indy1_tcp", face_name="brush_face", ext_off=0.032, tool_dim=(0.08, 0.32))
gscene.show_pose(crob.home_pose)

HOLD_NAME = "holder"
HOLD_HEIGHT = 0.4
HOLD_KEY = 1
front_dist = 0.45+CLEARANCE
holder_g = gscene.create_safe(GEOTYPE.SPHERE, HOLD_NAME, link_name=HOLD_LINK,dims=(0.01,0.01,0.01),
                           center=(front_dist, 0, HOLD_HEIGHT), rpy=(0,np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5))
holder_b = pscene.create_binder(HOLD_NAME, HOLD_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=HOLD_KEY)
floor_b = pscene.create_binder("floor_ws", "floor_ws", PlaceFrame, point=(0,0,WALL_THICKNESS/2))

## Table

In [6]:
TABLE_DIM = (0.6, 1.2, 0.7)
TABLE_NAME = "table"
TABLE_LOC = (2, 0, TABLE_DIM[2]/2)
TABLE_RPY = (0,0,0)
TABLE_KEY = 10

In [7]:
table_g = gscene.create_safe(GEOTYPE.BOX, TABLE_NAME, link_name="base_link", dims=TABLE_DIM,
                             center=TABLE_LOC, rpy=TABLE_RPY,
                             fixed=True, collision=True, color=(0.8,0.8,0.8,1))
table_b = pscene.create_binder(TABLE_NAME, TABLE_NAME, PlaceFrame, point=(0,0,TABLE_DIM[2]/2), key=TABLE_KEY)

## Waypoints

In [8]:
WP_NAME = "wp_{}"
WP_DIM = (0.1,0.1,0.01)
WP_LOC = (-0.2, 0, TABLE_DIM[2]/2+WP_DIM[2]/2)
WP_RPY = (0,0,0)

In [9]:
wp_g = gscene.create_safe(GEOTYPE.BOX, WP_NAME.format(0), link_name="base_link", dims=WP_DIM,
                          center=WP_LOC, rpy=WP_RPY,
                          fixed=True, collision=True, color=(0.0,0.0,0.8,1), parent=TABLE_NAME)
waypoint_s = pscene.create_subject(oname="waypoints", gname=TABLE_NAME, _type=WaypointTask,
                                 action_points_dict={
                                     wp_g.name: WayFrame(
                                         wp_g.name, wp_g, [0, 0, wp_g.dims[2] / 2], [0, 0, 0])})

## Bag

In [10]:
BAG_NAME = "bag"
BAG_DIM = (0.4, 0.29,0.635)

In [11]:
bag_g = gscene.create_safe(GEOTYPE.BOX, BAG_NAME, link_name="base_link",dims=BAG_DIM,
                           center=(1,0,BAG_DIM[2]/2), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.8,0.8,0.8,1))
handle = gscene.create_safe(GEOTYPE.BOX, BAG_NAME+"_hdl", link_name="base_link",dims=(0.2,0.03,0.02),
                           center=(0,0,BAG_DIM[2]/2+0.01), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.6,0.6,0.6,1), parent=BAG_NAME)

push_point_list = []
for i in range(4):
    R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
    point = np.round(np.multiply(BAG_DIM, -R[:,2])/2, 4)
    point[2] = -BAG_DIM[2]/2 + HOLD_HEIGHT
    ap_name = "side{}".format(i)
    push_point_list.append(FramePoint(ap_name, bag_g, point=point, rpy=Rot2rpy(R), key=HOLD_KEY))

slide_point = SlidePoint("bottom_p", bag_g, point=(0,0,-BAG_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                                           binded_on=floor_b, dist_push=0.5)

bag_s = pscene.create_subject(oname=BAG_NAME, gname=BAG_NAME, _type=PushObject, 
                              push_point_list=push_point_list, slide_point=slide_point)

### prepare planner

In [12]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [13]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=30)
tplan.prepare()

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [14]:
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan.motion_filters = [GraspChecker(pscene)]
mplan.incremental_constraint_motion = True

### Plan

In [15]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['bag', 'waypoints']
('floor_ws', 0)


In [16]:
goal_nodes = [('floor_ws',1)]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=False, add_homing=True)

try: 0 - ('floor_ws', 0)->('floor_ws', 1)
error: 1.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 0 - ('floor_ws', 0)->('holder', 0)
error: 2.3
try transition motion
transition motion tried: True
result: 0 - ('floor_ws', 0)->('holder', 0) = success
branching: 0->1 (0.78/100.0 s, steps/err: 18(216.26496315 ms)/0.00125286986031)
try: 0 - ('floor_ws', 0)->('floor_ws', 1)
error: 1.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 0 - ('floor_ws', 0)->('holder', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 0)->('holder', 0) = fail
try: 0 - ('floor_ws', 0)->('holder', 0)
error: 0.35
try transition motion
transition motion tried: True
result: 0 - ('floor_ws', 0)->('holder', 0) = success
branching: 0->2 (1.45/100.0 s, steps/err: 3(102.718114853 ms)/0.00145404485823)
try: 1 - ('holder', 0)->('floor_ws', 0)
error:

transition motion tried: False
Motion Plan Failure
result: 12 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 4 - ('floor_ws', 0)->('floor_ws', 1)
error: 0.6
try transition motion
transition motion tried: False
Motion Plan Failure
result: 4 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 10 - ('holder', 0)->('holder', 1)
error: 2.1
try transition motion
transition motion tried: False
Motion Plan Failure
result: 10 - ('holder', 0)->('holder', 1) = fail
try: 8 - ('floor_ws', 0)->('floor_ws', 1)
error: 2.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 8 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 2 - ('holder', 0)->('holder', 1)
error: 0.96
try transition motion
transition motion tried: False
Motion Plan Failure
result: 2 - ('holder', 0)->('holder', 1) = fail
try: 5 - ('floor_ws', 0)->('holder', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 5 - ('floor_ws', 0)->('holder', 0) = fail
try: 2 - ('holder', 0)-

Motion Plan Failure
result: 18 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 4 - ('floor_ws', 0)->('floor_ws', 1)
error: 0.6
try transition motion
transition motion tried: False
Motion Plan Failure
result: 4 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 15 - ('holder', 0)->('floor_ws', 0)
error: 0.34
end
constrained motion tried: True
result: 15 - ('holder', 0)->('floor_ws', 0) = success
branching: 15->25 (24.81/100.0 s, steps/err: 35(123.680114746 ms)/1.41301218057e-07)
try: 25 - ('floor_ws', 0)->('floor_ws', 1)
error: 2.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 25 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 14 - ('holder', 0)->('floor_ws', 0)
error: 0.3
end
constrained motion tried: True
result: 14 - ('holder', 0)->('floor_ws', 0) = success
branching: 14->26 (25.5/100.0 s, steps/err: 31(134.827852249 ms)/1.2994608082e-07)
try: 26 - ('floor_ws', 0)->('floor_ws', 1)
error: 0.66
try transition motion
transition motion tried: False
Motion Pla

Motion Plan Failure
result: 23 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 21 - ('holder', 0)->('holder', 1)
error: 2.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 21 - ('holder', 0)->('holder', 1) = fail
try: 11 - ('floor_ws', 0)->('holder', 0)
error: 2.2
try transition motion
transition motion tried: True
result: 11 - ('floor_ws', 0)->('holder', 0) = success
branching: 11->35 (41.69/100.0 s, steps/err: 25(510.742902756 ms)/0.00107882282827)
try: 6 - ('floor_ws', 0)->('holder', 0)
error: 0.0014
try transition motion
transition motion tried: True
result: 6 - ('floor_ws', 0)->('holder', 0) = success
branching: 6->36 (41.8/100.0 s, steps/err: 2(104.061841965 ms)/0.000877170299799)
try: 15 - ('holder', 0)->('floor_ws', 0)
error: 0.39
end
constrained motion tried: True
result: 15 - ('holder', 0)->('floor_ws', 0) = success
branching: 15->37 (41.94/100.0 s, steps/err: 40(139.056921005 ms)/1.43519290359e-07)
try: 37 - ('floor_ws', 0)->('floor_ws', 1)
er

In [17]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [18]:
snode_schedule = tplan.get_best_schedule(at_home=True)
ppline.play_schedule(snode_schedule)

('floor_ws', 0)->('holder', 0)
('holder', 0)->('floor_ws', 0)
('floor_ws', 0)->('holder', 0)
('holder', 0)->('floor_ws', 0)
('floor_ws', 0)->('floor_ws', 1)
('floor_ws', 1)->('floor_ws', 1)
('floor_ws', 1)->('floor_ws', 1)
